# EBUcore Metadata example

In [1]:
import sys
from tqdm.notebook import tnrange
from sgqlc.operation import Operation
from pdp_schema import pdp_schema
from ebucore_schema import ebucore_schema
from pdp_graphql_client_python import client

## [on hold] part 1: retrieve ids from INT using the pdp_schema (FARO)

In [2]:
# set endpoint to old PDP (FARO) schema on INT
%set_env PDP_API=https://graphql-api.pdp.int.srgssr.ch/graphql

env: PDP_API=https://graphql-api.pdp.int.srgssr.ch/graphql


In [3]:
# fetch environment variable with PDP endpoint API address
! echo "${PDP_API}"

https://graphql-api.pdp.int.srgssr.ch/graphql


In [4]:
op = Operation(pdp_schema.Query)
faro = op.random_faro_items()

In [5]:
faro.program_id()

programId

In [6]:
op

query {
  randomFaroItems {
    programId
  }
}

In [7]:
# fetch environment variable with PDP endpoint API address
! echo "${PDP_API}"

https://graphql-api.pdp.int.srgssr.ch/graphql


In [8]:
# download data
page_requests = 1
faro_items = []
for i in tnrange(page_requests, desc="Downloading data"):
    # query page
    data = client.run_query(op)
    
    # add results
    faro_items += (op + data).random_faro_items

## part 2: use ids to query ebucore schema

In [9]:
# set endpoint to new EBUcore schema on DEV
%set_env PDP_API=https://graphql-api.pdp.dev.srgssr.ch/graphql

env: PDP_API=https://graphql-api.pdp.dev.srgssr.ch/graphql


In [10]:
# random program_ids are currently not supported in DEV
# op = Operation(ebucore_schema.Query)
# assets = op.assets(ids=[x.program_id for x in faro_items])

In [11]:
# use example
op = Operation(ebucore_schema.Query)
assets = op.assets(ids=[
    "30115005-A6C3-4708-98F8-10FB082E381E",
    "7296F1FD-5767-4BB9-9C3C-546959723141",
    "urn:srf:video:271310e9-f391-4d28-8495-be660fce42f1"])

In [12]:
op

query {
  assets(ids: ["30115005-A6C3-4708-98F8-10FB082E381E", "7296F1FD-5767-4BB9-9C3C-546959723141", "urn:srf:video:271310e9-f391-4d28-8495-be660fce42f1"]) {
    assetId
    title
    abstract
    date
    hasContributor {
      agentName
      hasRole
    }
  }
}

### Define fragment queries with `__as__` statement

In [13]:
# define fragements
from ebucore_schema import Series, Episode, Staff

assets.asset_id()
assets.title()
assets.__as__(Series).total_number_of_episodes() # => if asset is of instance Series, retrieve the number of episodes
assets.__as__(Episode).orientation()
assets.has_contributor.__as__(Staff).given_name()
assets.has_contributor.__as__(Staff).family_name()

familyName

In [14]:
op

query {
  assets(ids: ["30115005-A6C3-4708-98F8-10FB082E381E", "7296F1FD-5767-4BB9-9C3C-546959723141", "urn:srf:video:271310e9-f391-4d28-8495-be660fce42f1"]) {
    assetId
    title
    __typename
    hasContributor {
      __typename
      ... on Staff {
        givenName
        familyName
      }
    }
    ... on Series {
      totalNumberOfEpisodes
    }
    ... on Episode {
      orientation
    }
  }
}

In [15]:
# query items
data = client.run_query(op)

In [16]:
data

{'data': {'assets': [{'assetId': '30115005-a6c3-4708-98f8-10fb082e381e',
    '__typename': 'Episode',
    'hasContributor': [{'__typename': 'Staff',
      'givenName': 'Béatrice',
      'familyName': 'Mohr'},
     {'__typename': 'Staff', 'givenName': 'Kurt', 'familyName': 'Schaad'}],
    'orientation': None,
    'title': 'Fernweh: Von Kapstadt zu den Viktoriafällen'},
   {'assetId': '7296f1fd-5767-4bb9-9c3c-546959723141',
    '__typename': 'Series',
    'title': 'Die Strassenfliege',
    'totalNumberOfEpisodes': '5',
    'hasContributor': []},
   {'assetId': 'urn:srf:video:271310e9-f391-4d28-8495-be660fce42f1',
    '__typename': 'Episode',
    'hasContributor': [{'__typename': 'Staff',
      'givenName': 'Ueli',
      'familyName': 'Schmezer'}],
    'orientation': None,
    'title': 'Billiger Echtpelz. Knausrige Samsung. Winterhandschuh-Test.'}]}}

In [17]:
results = (op + data).assets

In [18]:
results

[Episode(asset_id='30115005-a6c3-4708-98f8-10fb082e381e', title='Fernweh: Von Kapstadt zu den Viktoriafällen', __typename__='Episode', has_contributor=[Staff(__typename__='Staff', given_name='Béatrice', family_name='Mohr'), Staff(__typename__='Staff', given_name='Kurt', family_name='Schaad')], orientation=None),
 Series(asset_id='7296f1fd-5767-4bb9-9c3c-546959723141', title='Die Strassenfliege', __typename__='Series', has_contributor=[], total_number_of_episodes='5'),
 Episode(asset_id='urn:srf:video:271310e9-f391-4d28-8495-be660fce42f1', title='Billiger Echtpelz. Knausrige Samsung. Winterhandschuh-Test.', __typename__='Episode', has_contributor=[Staff(__typename__='Staff', given_name='Ueli', family_name='Schmezer')], orientation=None)]

In [19]:
for result in results:
    if isinstance(result, Episode):
        contributor_names = ','.join([x.given_name+' '+x.family_name for x in result.has_contributor])
        print(f'Episode "{result.title}" has contributors {contributor_names})')
    if isinstance(result, Series):
        print(f'Series "{result.title}" has {result.total_number_of_episodes} episodes')

Episode "Fernweh: Von Kapstadt zu den Viktoriafällen" has contributors Béatrice Mohr,Kurt Schaad)
Series "Die Strassenfliege" has 5 episodes
Episode "Billiger Echtpelz. Knausrige Samsung. Winterhandschuh-Test." has contributors Ueli Schmezer)


### load into pandas

In [20]:
import pandas as pd

In [21]:
df = pd.DataFrame([x.__json_data__ for x in results])

In [22]:
df

,assetId,__typename,hasContributor,orientation,title,totalNumberOfEpisodes
0,30115005-a6c3-4708-98f8-10fb082e381e,Episode,"[{'givenName': 'Béatrice', 'familyName': 'Mohr...",NaN,Fernweh: Von Kapstadt zu den Viktoriafällen,NaN
1,7296f1fd-5767-4bb9-9c3c-546959723141,Series,[],NaN,Die Strassenfliege,5
2,urn:srf:video:271310e9-f391-4d28-8495-be660fce...,Episode,"[{'givenName': 'Ueli', 'familyName': 'Schmezer'}]",NaN,Billiger Echtpelz. Knausrige Samsung. Winterha...,NaN


In [23]:
# explode has_Contributor array
df2 = df.explode('hasContributor')

In [24]:
df2

,assetId,__typename,hasContributor,orientation,title,totalNumberOfEpisodes
0,30115005-a6c3-4708-98f8-10fb082e381e,Episode,"{'givenName': 'Béatrice', 'familyName': 'Mohr'}",NaN,Fernweh: Von Kapstadt zu den Viktoriafällen,NaN
0,30115005-a6c3-4708-98f8-10fb082e381e,Episode,"{'givenName': 'Kurt', 'familyName': 'Schaad'}",NaN,Fernweh: Von Kapstadt zu den Viktoriafällen,NaN
1,7296f1fd-5767-4bb9-9c3c-546959723141,Series,NaN,NaN,Die Strassenfliege,5
2,urn:srf:video:271310e9-f391-4d28-8495-be660fce...,Episode,"{'givenName': 'Ueli', 'familyName': 'Schmezer'}",NaN,Billiger Echtpelz. Knausrige Samsung. Winterha...,NaN


In [25]:
pd.json_normalize(data = df2.loc[df2.__typename == 'Episode'].hasContributor, meta=['assetId'])

,givenName,familyName
0,Béatrice,Mohr
1,Kurt,Schaad
2,Ueli,Schmezer
